<a href="https://colab.research.google.com/github/KingT5M/NARENDRA-SINGLE-FAULT/blob/main/NARENDRA_SINGLE_FAULT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# Import packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import kerastuner as kt
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv1D, BatchNormalization, MaxPooling1D, LSTM, Flatten, Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import Callback

In [28]:
# Set paths
data_dir = r'C:\Users\T5M\Desktop\NARENDRA-SINGLE FAULT\FAULT DATASET'
graphs_path = os.path.join(data_dir, 'graphs')
best_model_path = os.path.join(data_dir, 'best_model')

# Create directories if they don't exist
os.makedirs(graphs_path, exist_ok=True)
os.makedirs(best_model_path, exist_ok=True)

# Read and process data
file_paths = [
    'rec3_002-rpm-healthy.csv',
    'rec3_004-gain-2.csv',
    'rec3_007-offset-1000.csv',
    'rec3_008-stuck.csv',
    'rec3_009-noise.csv',
    'rec3_010-drift.csv',
    'rec3_010-hard-over.csv',
    'rec3_012-delay-2.0.csv',
    'rec3_012-spike.csv'
]

def process_file(file_path):
    df = pd.read_csv(os.path.join(data_dir, file_path))
    df['time'] = df['time'].round(3)
    threshold_index = df[df['time'] > 300.66].index.min()
    if not pd.isnull(threshold_index):
        df = df.loc[:threshold_index]
    df.to_csv(os.path.join(data_dir, file_path), index=False)

for file_path in file_paths:
    process_file(file_path)

# Concatenate data
dfs = [pd.read_csv(os.path.join(data_dir, file_path)) for file_path in file_paths]
concatenated_df = pd.concat(dfs, ignore_index=True).dropna().sample(frac=1).reset_index(drop=True)

# Split data
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

train_end = int(len(concatenated_df) * train_ratio)
val_end = int(len(concatenated_df) * (train_ratio + val_ratio))

train_df = concatenated_df[:train_end]
val_df = concatenated_df[train_end:val_end]
test_df = concatenated_df[val_end:]

# Extract features and labels
def extract_features_labels(df):
    X = df.iloc[:, :2].values
    y = df.iloc[:, 2].values
    return X, y

X_train, y_train = extract_features_labels(train_df)
X_val, y_val = extract_features_labels(val_df)
X_test, y_test = extract_features_labels(test_df)

# Normalize features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y_train_encoded = encoder.fit_transform(y_train.reshape(-1, 1))
y_val_encoded = encoder.transform(y_val.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.reshape(-1, 1))

# Create sequences of 30
def create_sequences(X, y):
    sequence_length = 30
    X_sequences = [X[i:i+sequence_length] for i in range(len(X) - sequence_length + 1)]
    y_sequences = [y[i+sequence_length-1] for i in range(len(y) - sequence_length + 1)]
    return np.array(X_sequences), np.array(y_sequences)

X_train_array, y_train_array = create_sequences(X_train_scaled, y_train_encoded)
X_val_array, y_val_array = create_sequences(X_val_scaled, y_val_encoded)
X_test_array, y_test_array = create_sequences(X_test_scaled, y_test_encoded)

In [34]:
# Define metrics callback
class MetricsCallback(Callback):
    def __init__(self, validation_data, graphs_path):
        super(MetricsCallback, self).__init__()
        self.validation_data = validation_data
        self.graphs_path = graphs_path

    def on_epoch_end(self, epoch, logs=None):
        X_val, y_val = self.validation_data
        y_pred = self.model.predict(X_val)
        y_pred_classes = tf.argmax(y_pred, axis=1)
        y_true = tf.argmax(y_val, axis=1)

        precision = precision_score(y_true, y_pred_classes, average='weighted')
        recall = recall_score(y_true, y_pred_classes, average='weighted')
        f1 = f1_score(y_true, y_pred_classes, average='weighted')

        print(f"Validation Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

        if precision >= 0.9886 and recall >= 0.9890 and f1 >= 0.9888:
            print("Achieved desired metrics. Stopping training.")
            self.model.stop_training = True
            
            # Sensor fault types
            fault_types = ['healthy', 'gain', 'offset', 'stuck-at', 'noise', 'drift', 'hard-over', 'delay-time', 'spike']

            # Plotting precision, recall, and F1-score
            plt.figure(figsize=(10, 5))
            bar_width = 0.2
            index = list(range(len(fault_types)))
            plt.bar(index, precision, bar_width, label='Precision')
            plt.bar([i + bar_width for i in index], recall, bar_width, label='Recall')
            plt.bar([i + 2 * bar_width for i in index], f1, bar_width, label='F1-score')
            plt.xlabel('Fault Types')
            plt.ylabel('Scores')
            plt.title('Precision, Recall, and F1-score')
            plt.xticks([i + bar_width for i in index], fault_types, rotation=45)
            plt.legend()
            plt.tight_layout()
            plt.grid(axis='y')
            plt.savefig(os.path.join(self.graphs_path, 'metrics_combined.png'))
            plt.close()

# Define hypermodel
def build_model(hp):
    model = Sequential()
    
    cnn_layers = hp.Int('cnn_layers', min_value=0, max_value=5, default=5)
    lstm_layers = hp.Int('lstm_layers', min_value=0, max_value=5, default=4)
    dense_layers = hp.Int('dense_layers', min_value=0, max_value=5, default=0)
    epochs = hp.Int('epochs', min_value=50, max_value=900, default=850)
    max_pooling = hp.Int('max_pooling', min_value=0, max_value=1, default=1)
    dropout = hp.Int('dropout', min_value=0, max_value=2, default=0)
    batch_norm = hp.Int('batch_norm', min_value=0, max_value=2, default=2)
    batch_size = hp.Int('batch_size', min_value=64, max_value=150, default=64)
    learning_rate = hp.Float('learning_rate', min_value=0.0001, max_value=0.001, default=0.0005, sampling='linear')

    for i in range(cnn_layers):
        model.add(Conv1D(filters=8, kernel_size=2, activation='relu', padding='same', input_shape=(30, 2)))
        if batch_norm:
            model.add(BatchNormalization())

    if max_pooling:
        model.add(MaxPooling1D(pool_size=2))

    for i in range(lstm_layers):
        model.add(LSTM(units=64, activation='relu', return_sequences=True))
        if batch_norm:
            model.add(BatchNormalization())

    model.add(Flatten())

    for i in range(dense_layers):
        model.add(Dense(units=64, activation='relu'))
        if dropout:
            model.add(Dropout(0.5))
        if batch_norm:
            model.add(BatchNormalization())

    model.add(Dense(units=9, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Define ALL_CLASSES
ALL_CLASSES = {
    'Activation': Activation,
    'Conv1D': Conv1D,
    'BatchNormalization': BatchNormalization,
    'MaxPooling1D': MaxPooling1D,
    'LSTM': LSTM,
    'Flatten': Flatten,
    'Dense': Dense,
    'Dropout': Dropout
}

# Define REINFORCE algorithm
def reinforce_update(current_hp, rewards, learning_rate=0.001):
    rewards -= np.mean(rewards)
    rewards /= np.std(rewards)
    
    for hp_name, reward in zip(current_hp.keys(), rewards):
        current_hp[hp_name] += learning_rate * reward
    
    return current_hp

# Define parameters
num_episodes = 100
max_steps_per_episode = 50
current_hyperparameters = {
    'cnn_layers': 5,
    'lstm_layers': 4,
    'dense_layers': 0,
    'epochs': 850,
    'max_pooling': 1,
    'dropout': 0,
    'batch_norm': 2,
    'batch_size': 64,
    'learning_rate': 0.0005
}

# Define metrics callback
metrics_callback = MetricsCallback((X_val_array, y_val_array), graphs_path)

# Training loop with REINFORCE
for episode in range(num_episodes):
    episode_rewards = []
    
    for step in range(max_steps_per_episode):
        sampled_hyperparameters = {
            'cnn_layers': int(np.random.normal(current_hyperparameters['cnn_layers'], 0.1)),
            'lstm_layers': int(np.random.normal(current_hyperparameters['lstm_layers'], 0.1)),
            'dense_layers': int(np.random.normal(current_hyperparameters['dense_layers'], 0.1)),
            'epochs': int(np.random.normal(current_hyperparameters['epochs'], 0.1)),
            'max_pooling': int(np.random.normal(current_hyperparameters['max_pooling'], 0.1)),
            'dropout': int(np.random.normal(current_hyperparameters['dropout'], 0.1)),
            'batch_norm': int(np.random.normal(current_hyperparameters['batch_norm'], 0.1)),
            'batch_size': int(np.random.normal(current_hyperparameters['batch_size'], 0.1)),
            'learning_rate': np.random.normal(current_hyperparameters['learning_rate'], 0.1)
        }
        print("sampled_hyperparameters:", sampled_hyperparameters)

        # Debugging: Print ALL_CLASSES and sampled_hyperparameters
        print("ALL_CLASSES:", ALL_CLASSES)
        print("sampled_hyperparameters:", sampled_hyperparameters)

        # Create a new instance of HyperParameters
        hp = kt.HyperParameters()

        # Add configurations to the HyperParameters object
        for key, value in sampled_hyperparameters.items():
            hp.Fixed(key, value)

        # Call from_config() method with the configured HyperParameters object
        hp_config = {'space': sampled_hyperparameters}
        print("config after from_config:", hp_config)

        # Use the configured HyperParameters object when calling build_model()
        model = build_model(hp)
        
        print("Model built:", model)
        
        history = model.fit(
            X_train_array,
            y_train_array,
            validation_data=(X_val_array, y_val_array),
            batch_size=sampled_hyperparameters['batch_size'],
            epochs=sampled_hyperparameters['epochs'],
            verbose=0
        )
        
        y_pred = model.predict(X_val_array)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = np.argmax(y_val_array, axis=1)
        precision = precision_score(y_true, y_pred_classes, average='weighted')
        recall = recall_score(y_true, y_pred_classes, average='weighted')
        f1 = f1_score(y_true, y_pred_classes, average='weighted')
        reward = (precision >= 0.9886 and recall >= 0.9890 and f1 >= 0.9888) - 1
        
        episode_rewards.append(reward)
    
    current_hyperparameters = reinforce_update(current_hyperparameters, episode_rewards)
    print(f"Episode {episode + 1}: Rewards - {np.mean(episode_rewards)}")


# Get the best hyperparameters
best_hps = kt.HyperParameters().from_config(current_hyperparameters)

# Build the model with the best hyperparameters
best_model = build_model(best_hps)

# Train the model
history = best_model.fit(
    X_train_array,
    y_train_array,
    validation_data=(X_val_array, y_val_array),
    batch_size=best_hps.get('batch_size'),
    epochs=best_hps.get('epochs'),
    callbacks=[metrics_callback]
)

# Save the best model
best_model.save(os.path.join(best_model_path, 'best_model.h5'))

# Calculate metrics on the validation set
y_pred = best_model.predict(X_val_array)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val_array, axis=1)

precision = precision_score(y_true, y_pred_classes, average=None, zero_division=1)
recall = recall_score(y_true, y_pred_classes, average=None)
f1 = f1_score(y_true, y_pred_classes, average=None)

# Sensor fault types
fault_types = ['healthy', 'gain', 'offset', 'stuck-at', 'noise', 'drift', 'hard-over', 'delay-time', 'spike']

# Plotting precision, recall, and F1-score
plt.figure(figsize=(10, 5))
bar_width = 0.2
index = list(range(len(fault_types)))
plt.bar(index, precision, bar_width, label='Precision')
plt.bar([i + bar_width for i in index], recall, bar_width, label='Recall')
plt.bar([i + 2 * bar_width for i in index], f1, bar_width, label='F1-score')
plt.xlabel('Fault Types')
plt.ylabel('Scores')
plt.title('Precision, Recall, and F1-score')
plt.xticks([i + bar_width for i in index], fault_types, rotation=45)
plt.legend()
plt.tight_layout()
plt.grid(axis='y')
plt.savefig(os.path.join(graphs_path, 'metrics_combined.png'))
plt.close()

sampled_hyperparameters: {'cnn_layers': 5.189509646166705, 'lstm_layers': 4.005461764511269, 'dense_layers': -0.032989595386170065, 'epochs': 850.0048703204084, 'max_pooling': 1.0815271025526854, 'dropout': -0.1834531357665548, 'batch_norm': 2.2160481300151154, 'batch_size': 64.01970313687808, 'learning_rate': -0.025750921284782576}
ALL_CLASSES: {'Activation': <class 'keras.src.layers.core.activation.Activation'>, 'Conv1D': <class 'keras.src.layers.convolutional.conv1d.Conv1D'>, 'BatchNormalization': <class 'keras.src.layers.normalization.batch_normalization.BatchNormalization'>, 'MaxPooling1D': <class 'keras.src.layers.pooling.max_pooling1d.MaxPooling1D'>, 'LSTM': <class 'keras.src.layers.rnn.lstm.LSTM'>, 'Flatten': <class 'keras.src.layers.reshaping.flatten.Flatten'>, 'Dense': <class 'keras.src.layers.core.dense.Dense'>, 'Dropout': <class 'keras.src.layers.regularization.dropout.Dropout'>}
sampled_hyperparameters: {'cnn_layers': 5.189509646166705, 'lstm_layers': 4.005461764511269, 'd

TypeError: 'float' object cannot be interpreted as an integer